<a href="https://colab.research.google.com/github/shaon11579/DNN-fall-2021/blob/main/A3_Question_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
3 of 2
In [2]:
import time
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tensorflow.keras.applications.resnet50 import ResNet50
In [7]:
!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
--2021-10-12 18:15:57--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  88.6MB/s    in 2.7s    

2021-10-12 18:15:59 (88.6 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]

In [8]:
!unzip -q tiny-imagenet-200.zip && ls tiny-imagenet-200
test  train  val  wnids.txt  words.txt
In [3]:
def mapDirectoryToInt():
    ID_DICT = {}
    PATH = 'tiny-imagenet-200/wnids.txt'
    FILEOPEN = open(PATH,'r')
    for i, folder in enumerate(FILEOPEN):
        ID_DICT[folder.replace('\n','')]=i
    return ID_DICT
In [4]:
def getImage(folderName=None):
    DATASET_PATH = 'tiny-imagenet-200/'+folderName+'/'
    directories = os.listdir(DATASET_PATH)
    ONE_HOT_ENCODED = np.eye(len(directories))
    MAP_DIC = mapDirectoryToInt()
    TRAIN_IMAGES = []
    TRAIN_LABELS = []
    CLASS = None
    for directory in directories:
        path = DATASET_PATH+directory+'/'
        files = os.listdir(path)
        if files[0].endswith('.txt'):
            CLASS = files[0]
        else:
            CLASS = files[1]
        for file in files:
            if file.endswith('.txt'):
                continue
            IMG_FOLDER_PATH = path+file+'/'
            IMAGES = os.listdir(IMG_FOLDER_PATH)
            for IMAGE in IMAGES:
                IMG_PATH = IMG_FOLDER_PATH+IMAGE
                img = cv2.imread(IMG_PATH)
                if len(img.shape)< 3:
                    continue
                #img = np.reshape(img,[64,64,3])
                TRAIN_IMAGES.append(img)
                TRAIN_LABELS.append((np.reshape(ONE_HOT_ENCODED[MAP_DIC[CLASS.replace('_boxes.txt','')]], [200])))
    return np.array(TRAIN_IMAGES), np.array(TRAIN_LABELS)
In [5]:
def loadValidationData(folderName=None):
    DATASET_PATH = 'tiny-imagenet-200/'
    DATASET_FILE_PATH = 'tiny-imagenet-200/'+folderName+'/val_annotations.txt'
    ONE_HOT_ENCODED = np.eye(200)
    MAP_DIC = mapDirectoryToInt()
    file = open(DATASET_FILE_PATH)
    TEST = []
    TEST_LABELS = []
    for line in file:
        image, class_id = line.split('\t')[:2]
        IMG_PATH = DATASET_PATH+folderName+'/images/'+image
        img = cv2.imread(IMG_PATH)
        if len(img.shape) <3:
            continue
        TEST.append(img)
        TEST_LABELS.append((np.reshape(ONE_HOT_ENCODED[MAP_DIC[class_id]], [200])))
    return np.array(TEST), np.array(TEST_LABELS)
In [9]:
from tensorflow import keras
TEST_IMAGES, TEST_LABELS = loadValidationData('val')
TEST_IMAGES = keras.applications.resnet50.preprocess_input(TEST_IMAGES)
In [10]:
print(TEST_IMAGES.shape)
print(TEST_LABELS.shape)
(10000, 64, 64, 3)
(10000, 200)
In [11]:
TRAIN_IMAGES, TRAIN_LABELS = getImage('train')
In [12]:
TRAIN_IMAGES =  keras.applications.resnet50.preprocess_input(TRAIN_IMAGES)
In [13]:
path = 'tiny-imagenet-200/'
In [14]:
import tensorflow.keras as K
In [15]:
input_shape = K.Input(shape=(64,64,3))
In [16]:
resModel = ResNet50(include_top=False, weights='imagenet',input_tensor=input_shape)
#model.summary()
Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
94773248/94765736 [==============================] - 1s 0us/step
94781440/94765736 [==============================] - 1s 0us/step
In [17]:
for layer in resModel.layers[:-2]:
  layer.trainable = False
In [18]:
import tensorflow as tf 
model = K.models.Sequential()
model.add(resModel)
model.add(K.layers.Flatten())
model.add(K.layers.BatchNormalization())
model.add(K.layers.Dropout(0.7))

model.add(K.layers.Dense(200, activation='softmax'))
model.summary()
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
=================================================================
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 8192)              32768     
_________________________________________________________________
dropout (Dropout)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 200)               1638600   
=================================================================
Total params: 25,259,080
Trainable params: 1,654,984
Non-trainable params: 23,604,096
_________________________________________________________________
In [19]:
model.compile(loss='categorical_crossentropy',
                  optimizer=K.optimizers.Adam(learning_rate=0.0001),
                  metrics=['accuracy'])
In [20]:
TRAIN_IMAGES.shape, TRAIN_LABELS.shape, TEST_IMAGES.shape, TEST_LABELS.shape
Out[20]:
((100000, 64, 64, 3), (100000, 200), (10000, 64, 64, 3), (10000, 200))
In [21]:
history = model.fit(TRAIN_IMAGES, TRAIN_LABELS, batch_size=32,  epochs=30, verbose=1,
                        validation_data=(TEST_IMAGES, TEST_LABELS),shuffle=True)
Epoch 1/30
3125/3125 [==============================] - 77s 19ms/step - loss: 4.6520 - accuracy: 0.1947 - val_loss: 2.7350 - val_accuracy: 0.4032
Epoch 2/30
3125/3125 [==============================] - 51s 16ms/step - loss: 3.4539 - accuracy: 0.3289 - val_loss: 2.5844 - val_accuracy: 0.4296
Epoch 3/30
3125/3125 [==============================] - 51s 16ms/step - loss: 3.0541 - accuracy: 0.3728 - val_loss: 2.5127 - val_accuracy: 0.4423
Epoch 4/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.7906 - accuracy: 0.4051 - val_loss: 2.4784 - val_accuracy: 0.4534
Epoch 5/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.6056 - accuracy: 0.4270 - val_loss: 2.4419 - val_accuracy: 0.4600
Epoch 6/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.4798 - accuracy: 0.4479 - val_loss: 2.4455 - val_accuracy: 0.4586
Epoch 7/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.3696 - accuracy: 0.4624 - val_loss: 2.4313 - val_accuracy: 0.4594
Epoch 8/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.2899 - accuracy: 0.4742 - val_loss: 2.4287 - val_accuracy: 0.4625
Epoch 9/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.2243 - accuracy: 0.4849 - val_loss: 2.4265 - val_accuracy: 0.4661
Epoch 10/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.1680 - accuracy: 0.4913 - val_loss: 2.4176 - val_accuracy: 0.4639
Epoch 11/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.1342 - accuracy: 0.4986 - val_loss: 2.4273 - val_accuracy: 0.4675
Epoch 12/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.0954 - accuracy: 0.5037 - val_loss: 2.4291 - val_accuracy: 0.4667
Epoch 13/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.0689 - accuracy: 0.5075 - val_loss: 2.4365 - val_accuracy: 0.4654
Epoch 14/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.0446 - accuracy: 0.5118 - val_loss: 2.4270 - val_accuracy: 0.4711
Epoch 15/30
3125/3125 [==============================] - 51s 16ms/step - loss: 2.0235 - accuracy: 0.5131 - val_loss: 2.4260 - val_accuracy: 0.4694
Epoch 16/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.9957 - accuracy: 0.5207 - val_loss: 2.4439 - val_accuracy: 0.4671
Epoch 17/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.9973 - accuracy: 0.5196 - val_loss: 2.4507 - val_accuracy: 0.4665
Epoch 18/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.9765 - accuracy: 0.5251 - val_loss: 2.4461 - val_accuracy: 0.4692
Epoch 19/30
3125/3125 [==============================] - 52s 17ms/step - loss: 1.9625 - accuracy: 0.5252 - val_loss: 2.4562 - val_accuracy: 0.4704
Epoch 20/30
3125/3125 [==============================] - 54s 17ms/step - loss: 1.9514 - accuracy: 0.5271 - val_loss: 2.4617 - val_accuracy: 0.4689
Epoch 21/30
3125/3125 [==============================] - 52s 17ms/step - loss: 1.9363 - accuracy: 0.5318 - val_loss: 2.4579 - val_accuracy: 0.4678
Epoch 22/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.9345 - accuracy: 0.5304 - val_loss: 2.4562 - val_accuracy: 0.4671
Epoch 23/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.9189 - accuracy: 0.5327 - val_loss: 2.4652 - val_accuracy: 0.4666
Epoch 24/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.9106 - accuracy: 0.5317 - val_loss: 2.4557 - val_accuracy: 0.4689
Epoch 25/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.8997 - accuracy: 0.5378 - val_loss: 2.4617 - val_accuracy: 0.4691
Epoch 26/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.8935 - accuracy: 0.5381 - val_loss: 2.4566 - val_accuracy: 0.4670
Epoch 27/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.8875 - accuracy: 0.5396 - val_loss: 2.4587 - val_accuracy: 0.4696
Epoch 28/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.8868 - accuracy: 0.5389 - val_loss: 2.4741 - val_accuracy: 0.4703
Epoch 29/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.8678 - accuracy: 0.5410 - val_loss: 2.4749 - val_accuracy: 0.4643
Epoch 30/30
3125/3125 [==============================] - 51s 16ms/step - loss: 1.8739 - accuracy: 0.5396 - val_loss: 2.4777 - val_accuracy: 0.4675
In [22]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title("Model Accuracy(Fine-Tunining)")
plt.show()

In [23]:
plt.plot(history.history['loss'],'r', label='Training Loss')
plt.plot(history.history['val_loss'],'c', label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title("Model Loss(Fine-Tunining)")
plt.show()

In [27]:
print('Training accuracy: ', np.around(history.history['accuracy'][29]*100, decimals=2),'%')
print('Validaiton accuracy: ', np.around(history.history['val_accuracy'][29]*100, decimals=2),'%')
Training accuracy:  53.96 %
Validaiton accuracy:  46.75 %
In [28]:
prediction = model.evaluate(TEST_IMAGES,TEST_LABELS)
print("Testing Accuracy: ", np.around(prediction[1]*100, decimals=2))
print("Testing Loss: ", np.around(prediction[0],decimals=2))
313/313 [==============================] - 5s 16ms/step - loss: 2.4777 - accuracy: 0.4675
Testing Accuracy:  46.75
Testing Loss:  2.48